# <center>Big Data &ndash; Exercise 1</center>
## <center>Fall 2023 &ndash; Week 1 &ndash; ETH Zurich</center>

### Aims
- **After this exercise:** 
    - Understand the SQL language and its common query patterns.
    - Understand the 'table' data shape, normalization, and when they can (and should) be used.
    - be able to query data in tables with the SQL language.
- **Later in the semester:** 
    - Relate these language features and query patterns relative to other data shapes, technologies, and the languages designed to query them.
    - Understand when tables are not the appropriate shape for your data and when you can (and should) throw normalization away!

### Prerequisites
In this exercise, you will brush-up the fundamental concepts of relational databases and SQL. If you haven't taken an introductory databases course (or want to refresh your knowledge) we recommend you to read the following:

Garcia-Molina, Ullman, Widom: Database Systems: The Complete Book. Pearson, 2. Edition, 2008. (Chapters 1, 2, 3, and 6) [Available in the ETH Library] [[Online]](https://ebookcentral.proquest.com/lib/ethz/detail.action?pq-origsite=primo&docID=5832965) [[Selected solutions]](http://infolab.stanford.edu/~ullman/dscbsols/sols.html).

Or have a look at the recordings from Information Systems for Engineers - ETH Zurich, available on [[YouTube]](https://www.youtube.com/c/GhislainFournysLectures).

### Database Set-up
We will be once again working in the ExamMagicBox (you can find it in the following [[link]](http://please.add.link.here) if you have not downloaded it yet): please drag this Notebook in the folder. Just like last week, activate the docker container for the exercise sheet with `docker compose up`; please wait for the message `PostgreSQL init process complete; ready for start up` in the docker logs before proceeding! Alternatively you can start the Docker with `docker compose up -d` and wait for the command to execute: please note that you are creating the containers in the background this way. You can then type `docker compose down` when you are done.

As before, we set up our connection to the database and enable use of `%sql` and `%%sql`.

In [2]:
server='db'
user='postgres'
password='example'
database='postgres'
connection_string=f'postgresql://{user}:{password}@{server}:5432/{database}'

In [3]:
# This line tells Jupyter Notebook to reload the SQL extension. 
# The SQL extension in Jupyter allows you to run SQL queries directly in a Jupyter Notebook cell. 
%reload_ext sql
%sql $connection_string

In [4]:
%%sql
SELECT version();

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


version
"PostgreSQL 16.0 (Debian 16.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit"


In [8]:
%%sql
Select 
from artists
natural join released_by
natural join releases
where artist_id = '47742'

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


artist_id,name,realname,profile,url
47742,Diana Ross,Diane Ernestine Earle Ross,"Born March 26, 1944, Detroit, Michigan, USA. American vocalist, producer, and occasional actress. Working within groups early on in her musical career, she is equally known as an artist in her own right, and has released music in many genres; R&B, soul, disco, and jazz. Initial success came as lead singer of [a17966]. Sister to Motown songwriter [a=Arthur Ross].",http://www.myspace.com/dianaross


### Origin of the data
You can find more information on the dataset in the following links
- [Discogs](https://www.discogs.com/)
- [Discogs XML data dumps](http://data.discogs.com/)

If you do not want to use Docker or it does not work you can download the dataset from this [link](https://cloud.inf.ethz.ch/s/DtjCHTLRHT39BRN/download/discogs.dump.xz), see `postgres-init.sh` to see how to import it)

## Exercise 1: Explore the dataset
We want to first understand the dataset a bit better. You will find some queries below to help you explore the schema.

### List tables
The following query retrieves a list of tables in the database from a system table describing the current database.

In [8]:
%%sql 
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

 * postgresql://postgres:***@db:5432/postgres
22 rows affected.


table_name
companies
jobs
employees02
badges
comments
inventory
posthistory
postlinks
posts
tags


### List attributes/columns
The following query retrieves a list of columns from the tables in the database.

In [9]:
%%sql 
SELECT table_name, column_name, data_type, is_nullable, ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public'
AND table_name NOT LIKE 'pg_%'
ORDER BY table_name, ordinal_position;

 * postgresql://postgres:***@db:5432/postgres
128 rows affected.


table_name,column_name,data_type,is_nullable,ordinal_position
artists,artist_id,integer,NO,1
artists,name,character varying,YES,2
artists,realname,text,YES,3
artists,profile,text,YES,4
artists,url,text,YES,5
badges,id,integer,NO,1
badges,userid,integer,YES,2
badges,name,character varying,YES,3
badges,date,timestamp without time zone,YES,4
badges,class,smallint,YES,5


### Have a look at the datasets
The following simple query gives the first 5 rows of the `artists` dataset

In [10]:
%%sql
SELECT * FROM artists LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id,name,realname,profile,url
1,The Persuader,Jesper Dahlbäck,None,None
2,Mr. James Barth & A.D.,Cari Lekebusch & Alexi Delano,None,None
3,Josh Wink,Joshua Winkelman,"After forming [l=Ovum Recordings] as an independent label in October 1994 with former partner [a=King Britt], Josh recorded the cult classic 'Liquid Summer'. He went on to release singles for a wide variety of revered European labels ranging from Belgium's [l=R & S Records] to England's [l=XL Recordings]. In 1995, Wink became one of the first DJ-producers to translate his hard work into mainstream success when he unleashed a string of classics including 'Don't Laugh'¸ 'I'm Ready' and 'Higher State of Consciousness' that topped charts worldwide. More recently he has had massive club hits such as 'How's Your Evening So Far' and 'Superfreak' but he has also gained a lot of attention trough his remixes for [a=FC Kahuna], [a=Paul Oakenfold], [a=Ladytron], [a=Clint Mansell], [a=Sting] and [a=Depeche Mode], among others.",http://www.joshwink.com/
4,Johannes Heil,Johannes Heil,"Electronic music producer, musician and live performer, born 3 February 1978 near the town of Bad Nauheim, Germany. Founder of [l=JH] and [l=Metatron Recordings].",http://johannes-heil.com/
5,Heiko Laux,Heiko Laux,German DJ and producer based in Berlin. He is the founder of [l=Kanzleramt].,http://www.heiko-laux.com


Naturally we could write similar queries to better understand each of the other tables.

### With what you now know about the datasets, try to answer the following questions

1. Which concepts are modelled in the dataset and how do they relate to each other? <b>Hint</b>: how do the tables connect logically?
2. Why do you think this shape (table) was chosen for the data and why not the other shapes?
3. In which normal forms are the corresponding relations?
4. How can we denormalise the data to make some queries more efficient? <b>Hint</b>: have a look at the queries in the next session of the exercises to see if adding some columns to some tables could reduce the need to `JOIN`.
5. What potential problems could result from adding redundancy?

### Solution
1. The dataset contains data from the discogs. Artists, their details such as albums, salaries..., users of the platform...
2. We can define a Schema:
   - The concepts (tables like artists and salaries) are fixed and limited.
   - The attributes (columns like name and id) are also fixed and limited.
   - The attributes have fixed data types.
   - The different concepts are related to each other.
3. The relations are in 3NF:
   - All relations are tabular => 1NF
   - Relations have singular key ID => Attributes depends on all the prime attribtues => 2NF
   - No transitive dependencies => 3NF
4. If we merge the artists with releases and tracks, we wouldn't have to join them every time.
5. Insert and Update anomaly.

## Exercise 2: SQL warm-up
Let us begin with several SQL queries to ease us back into the language.

#### 1. Retrieve the artists with the name of 'Coldplay'.

In [17]:
%%sql 
SELECT * 
FROM artists 
WHERE name = 'Coldplay';

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


artist_id,name,realname,profile,url
29735,Coldplay,None,"Coldplay is an English rock band from London, England. They've been a band since January 16, 1998 when they lost a demotape competition on XFM in London. Philip Christopher Harvey is the band's manager. [b][u]Line-up:[/u][/b] Jonny Buckland (Jonathan Mark Buckland) - Guitar Will Champion (William Champion) - Drums Guy Berryman (Guy Rupert Berryman) - Bass Chris Martin (Christopher Anthony John Martin) - Vocals",http://coldplay.com/


#### 2. List the titles of all releases by that artist in alphabetical order.
<b>Hint</b>: Ignore the fact that different relases can have the same title.

In [21]:
%%sql
SELECT DISTINCT title
FROM artists 
JOIN released_by USING(artist_id)
JOIN releases USING(release_id)
WHERE name = 'Coldplay'
ORDER BY title;

 * postgresql://postgres:***@db:5432/postgres
40 rows affected.


title
Acoustic
A Rush Of Blood To The Head
Boot Of Sound
Brothers & Sisters
Clocks
Clocks...
Clocks / Chime Trance Remixes
Clocks (Cosmos Rmx)
Clocks (Dean Coleman Remix)
Clocks (Planet Rockers Remixes)


#### 3. How many tracks from 'Coldplay' have position '1'?

In [27]:
%%sql
SELECT COUNT(*)
FROM artists 
JOIN released_by USING(artist_id)
JOIN releases USING(release_id)
JOIN tracks USING(release_id)
WHERE name = 'Coldplay' AND position = '1';

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
32


#### 4. What is the average track duration?

In [29]:
%%sql
SELECT AVG(CAST(duration as FLOAT))
FROM tracks;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


avg
324.59572773212363


#### 5. How many artists have released tracks longer than twice the average?

In [54]:
%%sql
WITH average_duration AS (SELECT AVG(CAST(duration as FLOAT)) FROM tracks)

SELECT COUNT(DISTINCT artist_id)
FROM artists 
JOIN released_by USING(artist_id)
JOIN tracks USING(release_id)
WHERE duration > 2*(SELECT * FROM average_duration);

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
18894


## Exercise 3: more SQL
We will now see more complex SQL queries.

#### 1. What is the title of the album from 'Coldplay' with the most amount of tracks?

In [46]:
%%sql
WITH tracks_amount AS(SELECT releases.title AS album, COUNT(*) as number_of_tracks
                     FROM artists
                     JOIN released_by USING(artist_id)
                     JOIN releases USING(release_id)
                     JOIN tracks USING(release_id)
                     WHERE name = 'Coldplay'
                     GROUP BY releases.release_id)

SELECT album, number_of_tracks
FROM tracks_amount
WHERE number_of_tracks = (SELECT MAX(number_of_tracks)
                         FROM tracks_amount);

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


album,number_of_tracks
X&Y (Special Dutch Edition),19


#### 2. How many artists have released an album with total track duration above twice the average total track duration?

<b>Hint</b>: this is not the same as exercise 2.5 since we are lookong at the <b>total</b> track duration of the album.

In [53]:
%%sql
WITH album_duration AS (SELECT release_id, SUM(duration) as duration
                       FROM tracks
                       GROUP BY release_id),
average AS (SELECT AVG(duration)
           FROM album_duration)

SELECT COUNT(DISTINCT(artists.artist_id))
FROM artists
JOIN released_by USING(artist_id)
JOIN album_duration USING(release_id)
WHERE duration > 2 * (SELECT * from average)

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
16426


#### 3. How many artists have both a release with a track longer than average and one with total duration longer than average?

<b>Hint</b>: you can use `INTERSECT` or `EXISTS` to write your query.

In [55]:
%%sql
WITH average_track AS (SELECT AVG(CAST(duration as FLOAT)) FROM tracks),

album_duration AS (SELECT release_id, SUM(duration) as duration
                   FROM tracks
                   GROUP BY release_id),

album_average AS (SELECT AVG(duration) 
                  FROM album_duration),

artist_intersection AS (
    SELECT DISTINCT(artist_id)
    FROM artists
    JOIN released_by USING(artist_id)
    JOIN tracks USING(release_id)
    WHERE duration > 2 * (SELECT * FROM average_track)
    INTERSECT
    SELECT DISTINCT(artist_id)
    FROM artists
    JOIN released_by USING(artist_id)
    JOIN album_duration USING(release_id)
    WHERE duration > 2 * (SELECT * from album_average))

SELECT COUNT(*)
FROM artist_intersection;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
8146


#### 4. Show the artists have more than 200 releases in total but have no releases with the genre 'Pop' in reversed alphabetical order.

In [64]:
%%sql
WITH big_artists AS(
    SELECT artist_id, name, COUNT(*) AS count
    FROM artists 
    NATURAL JOIN released_by
    NATURAL JOIN releases
    GROUP BY artist_id
    HAVING COUNT(release_id)>200)

SELECT name, count
FROM big_artists
WHERE artist_id NOT IN (SELECT artist_id
                       FROM artists NATURAL JOIN released_by NATURAL JOIN releases
                       WHERE genre = 'Pop')
ORDER BY name DESC;

 * postgresql://postgres:***@db:5432/postgres
35 rows affected.


name,count
Yello,275
WestBam,236
Underworld,273
U2,365
The Shamen,221
The Prodigy,355
The Human League,210
The Cure,417
The Chemical Brothers,302
The Beatles,205


## Exercise 4: Discuss query patterns and language features of SQL
1. What patterns did you use in many of the queries above? 

2. What is the usual pattern of an SQL query? Which operations happen pre-grouping and which ones post-grouping?

3. What makes SQL a declarative language and what advantages does that have?

4. What makes SQL a functional language and what advantages does that have?

5. How would the denormalization we talked about previously simplify the queries?

### Solution
1. SELECT, PROJECT, JOIN, GROUP, ORDER.
2. The usual pattern is SELECT, FROM, JOIN, WHERE, GROUP BY, HAVING, ORDER BY, LIMIT, OFFSET. FROM and WHERE happen pre-grouping.
3. Declarative: we declare what we want and not how we want to get it. 
4. Functional: Because we can use result of SQL (With) as input to other queries.
5. No need to JOIN.

## Exercise 5: Limits of SQL (optional)
Explain what the following query does.
<b>Hints</b>: The query treats the data as if it was in graph shape.

In [65]:
%%sql
WITH RECURSIVE
    X AS (SELECT 3 AS Value),
    artist_releases AS (
        SELECT artists.artist_id, artists.name, releases.release_id, releases.title
        FROM artists, released_by, releases
        WHERE artists.artist_id = released_by.artist_id
        AND released_by.release_id = releases.release_id
    ),
    collaborations AS (
        SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
                ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
        FROM artist_releases AS ar1, artist_releases AS ar2
        WHERE ar1.release_id = ar2.release_id
        AND ar1.artist_id != ar2.artist_id
    ),
    X_hop_collaborations AS (
        SELECT * FROM collaborations  -- base case
        UNION
        SELECT c1.left_id, c1.left_name, c2.right_id, c2.right_name, c1.distance + 1 AS distance
        FROM X_hop_collaborations AS c1
        JOIN collaborations c2 ON c1.right_id = c2.left_id
        WHERE c1.distance < (SELECT * FROM X)
    )
SELECT * 
FROM X_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY distance, right_name;

 * postgresql://postgres:***@db:5432/postgres
146 rows affected.


left_id,left_name,right_id,right_name,distance
29735,Coldplay,124541,Bon Jovi,1
29735,Coldplay,1654,DK,1
29735,Coldplay,392179,G Synth,1
29735,Coldplay,10916,Jan Johnston,1
29735,Coldplay,12236,Marco V,1
29735,Coldplay,4213,Max Graham,1
29735,Coldplay,92827,Michelle Nicol,1
29735,Coldplay,1279,Orbital,1
29735,Coldplay,239326,Walsh & Coutre,1
29735,Coldplay,10785,Angelo Badalamenti,2


### Solution
